In [1]:
from genson import SchemaBuilder

DATASET = 'claude'

seed_schema_chatgpt = {
    'type': 'array',
    'items': {
        'type': 'object',
        'properties': {
            'mapping': {
                'type': 'object',
                'patternProperties': {
                    # UUID pattern - GenSON will fill in the actual schema
                    r'^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$': None,
                    # Also handle the client-created-root pattern
                    r'^client-created-': None
                }
            }
        }
    }
}

seed_schema_claude = {
    'type': 'array',
    'items': {
        'type': 'object',
        'properties': {
            'uuid': {
                'type': 'string',
                'pattern': r'^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$'
            },
            'name': {
                'type': 'string'
            },
            'created_at': {
                'type': 'string',
                'format': 'date-time'
            },
            'updated_at': {
                'type': 'string',
                'format': 'date-time'
            },
            'account': {
                'type': 'object',
                'properties': {
                    'uuid': {
                        'type': 'string',
                        'pattern': r'^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$'
                    }
                }
            },
            'chat_messages': {
                'type': 'array',
                'items': {
                    'type': 'object',
                    'properties': {
                        'uuid': {
                            'type': 'string',
                            'pattern': r'^[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}$'
                        },
                        'text': {
                            'type': 'string'
                        },
                        'content': {
                            'type': 'array',
                            'items': {
                                'type': 'object',
                                'properties': {
                                    'start_timestamp': {
                                        'type': 'string',
                                        'format': 'date-time'
                                    },
                                    'stop_timestamp': {
                                        'type': 'string',
                                        'format': 'date-time'
                                    },
                                    'type': {
                                        'type': 'string'
                                    },
                                    'text': {
                                        'type': 'string'
                                    },
                                    'citations': {
                                        'type': 'array'
                                    }
                                }
                            }
                        },
                        'sender': {
                            'type': 'string'
                        },
                        'created_at': {
                            'type': 'string',
                            'format': 'date-time'
                        },
                        'updated_at': {
                            'type': 'string',
                            'format': 'date-time'
                        },
                        'attachments': {
                            'type': 'array',
                            'items': {
                                'type': 'object',
                                'properties': {
                                    'file_name': {
                                        'type': 'string'
                                    },
                                    'file_size': {
                                        'type': 'integer'
                                    },
                                    'file_type': {
                                        'type': 'string'
                                    },
                                    'extracted_content': {
                                        'type': 'string'
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}

seed_schema = seed_schema_chatgpt
if DATASET == 'claude':
    seed_schema = seed_schema_claude
builder = SchemaBuilder()
builder.add_schema(seed_schema)

In [2]:
import sys
from pathlib import Path
import json
PATH=str((Path().cwd().parent /'src').absolute())
print(PATH)
if PATH not in sys.path:
    sys.path.append(PATH)

DATASET = 'claude'

/Users/dmarx/proj/chat2obs/src


In [3]:
root_chatgpt = "../data/ingestion/chatgpt/a40ff5f79c1b3edd3c366f0f628fb79170bae83ecf3a1758b5b258c71f843f53-2025-06-05-03-28-15-df2ed357a4e64443bf464446686c9692/"
root_claude = "/Users/dmarx/Downloads/data-2025-07-06-16-51-06"
root = root_chatgpt
if DATASET == 'claude':
    root = root_claude
fpath = Path(root) / "conversations.json"
convs = json.load(fpath.open())

In [4]:
convs[0].keys()

dict_keys(['uuid', 'name', 'created_at', 'updated_at', 'account', 'chat_messages'])

In [5]:
convs[-1].keys()

dict_keys(['uuid', 'name', 'created_at', 'updated_at', 'account', 'chat_messages'])

In [6]:
convs[0]['chat_messages']

[{'uuid': 'de06099d-65b6-48ea-8c3b-0591f4c1c106',
  'text': '',
  'content': [{'start_timestamp': '2024-09-25T03:10:44.580237Z',
    'stop_timestamp': '2024-09-25T03:10:44.580237Z',
    'type': 'text',
    'text': '',
    'citations': []}],
  'sender': 'human',
  'created_at': '2024-09-25T03:10:44.580237Z',
  'updated_at': '2024-09-25T03:10:44.580237Z',
  'attachments': [{'file_name': 'paste.txt',
    'file_size': 51684,
    'file_type': 'txt',
    'extracted_content': "summarize this talk for me:\n\nhello how are you nice to see you thank you to Defcon thank you especially to\n0:06\nthe goons and the other volunteers who are out here busting their humps so we can have such a great time uh and thank\n0:12\nyou all for coming uh I thought I had 50 minutes I have 45 I'm going to talk fast\n0:18\nand instead of Q&A there's a book seller in the book signing areas bringing in some of my books you don't have to buy a\n0:23\nbook just come over and say hello I'm going to sit there until every

In [7]:
from conversation_tagger import create_default_tagger

tagger = create_default_tagger(source=DATASET)
tagged_results = [tagger.tag_conversation(c) for c in convs]

In [8]:
for i, tr in enumerate(tagged_results):
    print(tr.annotations)
    if i > 10:
        break   


{'conversation_length': {'count': 1, 'category': 'single'}}
{'has_code_structure_patterns': True, 'first_user_has_large_content': True, 'conversation_length': {'count': 9, 'category': 'medium'}, 'conversation_has_code_structure_patterns': {'exchange_count': 9, 'total_exchanges': 9, 'percentage': 100.0}}
{'conversation_length': {'count': 4, 'category': 'medium'}}
{'has_code_structure_patterns': True, 'first_user_has_large_content': True, 'has_wiki_links': True, 'conversation_length': {'count': 24, 'category': 'long'}, 'conversation_has_code_structure_patterns': {'exchange_count': 16, 'total_exchanges': 24, 'percentage': 66.7}, 'conversation_has_wiki_links': {'exchange_count': 1, 'total_exchanges': 24, 'percentage': 4.2}}
{'has_code_structure_patterns': True, 'first_user_has_large_content': True, 'conversation_length': {'count': 5, 'category': 'medium'}, 'conversation_has_code_structure_patterns': {'exchange_count': 5, 'total_exchanges': 5, 'percentage': 100.0}}
{'conversation_length': {

In [9]:
tr.exchanges

[Exchange(exchange_id='23cc9901-b727-488e-913f-0db497549115', conversation_id='699ee842-1866-44cd-9a20-5be5eec04fe4', messages=[
 2024-10-23T21:07:34.105205Z - USER: Let's start with a workflow that builds the README dynamically from a jinja template that compiles sections of the readme dynamically from tempaltes defined for each chunk. starter sections: introduct..., 
 2024-10-23T21:07:34.105205Z - ASSISTANT: I'll help you create a workflow to dynamically generate a README using Jinja2 templates. I'll create a basic structure that can be easily extended.
 
 <antThinking>This is a good candidate for an artif...], annotations={}),
 Exchange(exchange_id='9258eb46-0e2c-4f2c-86a3-17b4aaaae7b7', conversation_id='699ee842-1866-44cd-9a20-5be5eec04fe4', messages=[
 2024-10-23T21:09:43.800394Z - USER: let's put the templates for this readme in docs/readme/, 
 2024-10-23T21:09:43.800394Z - ASSISTANT: I'll restructure the project to organize the README templates in the `docs/readme/` directory.
 

In [10]:
tr.exchanges[-1].messages[-1]
# TODO: parse out `<antThinking>` spans


2024-10-24T21:30:37.356343Z - ASSISTANT: Good call. Let's simplify by moving the features directly into the introduction template.

<antThinking>Need to remove features from config and place them directly in the introduction template, makin...

In [11]:
for i, conv in enumerate(tagged_results):
    if "cognitive bias" in conv.title.lower():# or conv.title is None or conv.title == "":
        print(conv.title)
        break


Cognitive Bias: Unanimity Illusion


In [12]:
conv.raw

{'uuid': 'a8fc92fc-8327-4d85-9914-33e172f88cae',
 'name': 'Cognitive Bias: Unanimity Illusion',
 'created_at': '2025-07-04T00:51:11.759990Z',
 'updated_at': '2025-07-04T00:57:16.321080Z',
 'account': {'uuid': '8690ea2d-a58d-429c-80b3-b07d63f1e6f3'},
 'chat_messages': [{'uuid': '28ed1053-c697-4c07-ace0-7c5467c21dfa',
   'text': 'the illusion of unanimity',
   'content': [{'start_timestamp': '2025-07-04T00:51:12.423043Z',
     'stop_timestamp': '2025-07-04T00:51:12.423043Z',
     'type': 'text',
     'text': 'the illusion of unanimity',
     'citations': []}],
   'sender': 'human',
   'created_at': '2025-07-04T00:51:12.428076Z',
   'updated_at': '2025-07-04T00:51:12.428076Z',
   'attachments': [],
   'files': []},
  {'uuid': '8c065913-18ba-4892-88d4-2b444d868b3a',
   'text': 'The user is asking me to create a wiki-style entry about "the illusion of unanimity" for a technical reference vault for an AI researcher, ML scientist, applied mathematician, and philosopher of mind. They want me t

In [13]:
project_entries="""
<ul class="flex flex-col gap-2"><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/16cee302-ae80-4a41-addc-d0f3627df7d9"><div class="w-full"><div class="truncate">Chernoff-Hoeffding Bound Explained</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">5 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/3d2b6b74-796b-49d5-a2c3-f52be67fde84"><div class="w-full"><div class="truncate">Percept-Action Loops and Thermodynamics</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">5 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/16141457-2b5a-4d5d-a2b8-667935bd8a04"><div class="w-full"><div class="truncate">Longman Defining Vocabulary</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">6 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/2aa2df8d-ef3d-4dd2-88ba-5a295aa788e2"><div class="w-full"><div class="truncate">Kelp Marine Biology Overview</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">6 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/97acc24e-84d7-4191-ae0e-c6b6172dd23d"><div class="w-full"><div class="truncate">Lost Media Mystery</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">6 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/236ed605-a780-4867-815a-de559f72f385"><div class="w-full"><div class="truncate">Memory Loss Through Doorways</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">6 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/57aa8f73-0df6-4b7f-bf50-d1c2c7560749"><div class="w-full"><div class="truncate">Replicator Dynamics Overview</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">6 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/520595fb-af47-4f2b-9faa-63c597389e6f"><div class="w-full"><div class="truncate">Internet Archive Data Security Risks</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">10 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/a8fc92fc-8327-4d85-9914-33e172f88cae"><div class="w-full"><div class="truncate">Cognitive Bias: Unanimity Illusion</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">10 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/287945f4-2640-43ef-998c-e634c19ffcc5"><div class="w-full"><div class="truncate">Gray Zone Strategic Tactics</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">10 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/e1b81fdd-e2de-4896-9d2c-f7c481006fd9"><div class="w-full"><div class="truncate">Technical Research Content Review</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">11 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/622c915f-8696-4b4d-9b38-7907af137ad4"><div class="w-full"><div class="truncate">Arctic Strategic Competition Framework</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">11 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/bdf8d687-8f6a-474e-9637-50b18cf36b4b"><div class="w-full"><div class="truncate">AI Operational Cadence</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">11 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/6d9022b5-15e0-4ba9-b546-b140301ca2d4"><div class="w-full"><div class="truncate">Random Matrix Theory Overview</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">12 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/9ae66745-2afb-4c78-9454-fb6223ebba25"><div class="w-full"><div class="truncate">Library of Alexandria Legacy</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">13 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/a48fdebd-79e1-4989-acc8-f452e4c57bab"><div class="w-full"><div class="truncate">Understanding Social Scripts</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">14 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/d8abe0c6-d630-402b-a0f1-bd7e0988c967"><div class="w-full"><div class="truncate">Wave Elongation Phenomenon</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">14 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/b8985c54-72f1-4b0b-8c80-057448445379"><div class="w-full"><div class="truncate">Bass Diffusion Model Overview</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">15 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/f9e91098-111f-42f1-afbe-ea0e67d15e6b"><div class="w-full"><div class="truncate">Higher-Order Thought Consciousness Theory</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">15 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/0fba8173-1c87-49cf-a61e-6757bb961fb2"><div class="w-full"><div class="truncate">Computational Resource Efficiency Credits</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">17 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/62e42488-8b00-4ba8-9cb1-6059b8548954"><div class="w-full"><div class="truncate">Pre-Columbian Amazonian Soil Engineering</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">17 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/8ab7345a-1e81-48c5-b6bf-dfbf4c073fec"><div class="w-full"><div class="truncate">Mental Masturbation and AI Interaction</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">17 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/78198cdd-7a66-4caf-99a4-9446ea446b13"><div class="w-full"><div class="truncate">Walsh System Overview</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">18 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/5d1b5368-9717-4db9-a57f-7e5a31b617a1"><div class="w-full"><div class="truncate">AI Reward Learning as Addiction</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">18 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/717f77fe-9583-4de2-8399-0a4cba3cb75c"><div class="w-full"><div class="truncate">Exploring Weltschmerz</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">20 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/2fa5894e-523d-4607-9944-42a9b82cea63"><div class="w-full"><div class="truncate">Jain-Krishna Model Research</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">21 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/5fd80dd6-a32e-4333-980f-49ce8a4da713"><div class="w-full"><div class="truncate">Substantial Similarity in Copyright Law</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">21 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/9b94ea9a-2594-48f2-b5a8-86ebb44d387a"><div class="w-full"><div class="truncate">Strategic Initiative Dynamics</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">22 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/8fcecd0f-193e-4b76-b760-649e0798cf8e"><div class="w-full"><div class="truncate">Classifier-Free Guidance Explained</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">22 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/5c212bbe-133f-4bb2-87fd-62f874acbd2b"><div class="w-full"><div class="truncate">Legal AI Precedent Landscape</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">22 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/baf72dfb-d608-4706-a116-260f07e1ccbd"><div class="w-full"><div class="truncate">Deceptive User Interface Design</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">24 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/7298ed78-7c73-43d9-9d28-6be2a7d42fae"><div class="w-full"><div class="truncate">Psychology of Risk Excitement</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">27 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/bb2ab335-44e5-43a4-9a65-dc690439b655"><div class="w-full"><div class="truncate">Variational Learning Techniques</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">29 days ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/83827b71-1abc-479f-b3f2-b99088dd4f19"><div class="w-full"><div class="truncate">Reasoning and Conditioning Concepts</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/6bb94b8a-e022-4736-90b6-c706b7eb0019"><div class="w-full"><div class="truncate">Nerode Equivalence Explained</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/76b51860-695d-4a31-a592-405529318d2f"><div class="w-full"><div class="truncate">Autocratic Breakthrough Dynamics</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/708584b1-2634-42e7-ab85-6f73d65e7261"><div class="w-full"><div class="truncate">Understanding Agnotology</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/37344bcf-87cf-44ee-a533-90171b64dba0"><div class="w-full"><div class="truncate">Organismic Systems Theory Framework</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/e0aeca54-de44-43e6-9012-bcd5f35180db"><div class="w-full"><div class="truncate">AI Safety vs User Autonomy</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/7fe2751e-d72b-4587-9587-36d6ee86c1a4"><div class="w-full"><div class="truncate">Protein Folding Geometric Modeling</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/e492fce5-f5c9-47ba-b7f7-f2ae3877f058"><div class="w-full"><div class="truncate">Untitled</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/6482c91e-50e7-4cea-aa76-40451ddf3faf"><div class="w-full"><div class="truncate">FSDP Distributed Training Overview</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/80bf551e-fcf9-42e3-9d2d-796a6fdf4e87"><div class="w-full"><div class="truncate">Uncomputation in Quantum Computing</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/8b5855d4-28de-4e44-8f64-8617e699cad4"><div class="w-full"><div class="truncate">Pharmacological Treatment for Gambling Addiction</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/34f0e0f6-939d-4bea-846c-ec2ef8ac5024"><div class="w-full"><div class="truncate">Sycophancy's Psychological Impact</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/25217557-a2eb-4a25-ae0d-3b59e93633d4"><div class="w-full"><div class="truncate">Shadow Lemma Concept</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/e0dfc197-1cb9-4e63-8b74-e7ccdc1b1e6e"><div class="w-full"><div class="truncate">Suppressing Passion Through Social Bullying</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/2ca046e4-6d55-4c0f-8974-cd241ab7121b"><div class="w-full"><div class="truncate">Negotiating Control Transfer</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/922b9909-c5b3-448c-be67-7a659c3c2b51"><div class="w-full"><div class="truncate">Shareholder Wealth Maximization Doctrine</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/5f561911-5fb4-4b8e-a51e-2582fe6e5df5"><div class="w-full"><div class="truncate">Identifying Wrong Abstractions in Software</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/7eda207b-a31f-4aff-8cf9-d4257a655879"><div class="w-full"><div class="truncate">Theories of Emotion in Cognitive Science</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/b8d9054b-ed04-4db1-9994-c2b1e2dd8bb5"><div class="w-full"><div class="truncate">Algorithmic Probability Fundamentals</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/9db81c8b-6163-43c9-9294-7044053a345b"><div class="w-full"><div class="truncate">McTaggart's Philosophical Theories</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/b717b32e-1145-4051-8f7f-d32ab97ea3c9"><div class="w-full"><div class="truncate">Chaitin's Omega: Fundamental Algorithmic Constant</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/aaf8b1a1-aac7-4fed-af23-cacc3cc02b59"><div class="w-full"><div class="truncate">Entropy and Negative Log-Likelihood Connection</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/9d435bea-fd88-4fa4-955f-0c019bd211e9"><div class="w-full"><div class="truncate">Framing the U.S. Constitution's Motivations</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/6ba50df3-5ea5-4820-b3d3-250d3236adf4"><div class="w-full"><div class="truncate">Misappropriation Theory in Securities Law</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/dc416ec6-989d-4636-afc2-b14121949ead"><div class="w-full"><div class="truncate">Mori-Zwanzig Formalism for Reduced Modeling</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/f61272cd-3975-44f8-90a0-e45af69016ad"><div class="w-full"><div class="truncate">Geometric Mechanics Framework</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/78a60f2d-56d8-4890-af4a-a095cd8cf68e"><div class="w-full"><div class="truncate">Principle of Subsidiarity</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/47b0b59c-13d2-4cb4-bd08-f8d71ff682f0"><div class="w-full"><div class="truncate">eukaryogenesis</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">1 month ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/1a1aa2bd-3f05-405d-97af-d8db7965ad06"><div class="w-full"><div class="truncate">Consolidating Power Through Institutional Capture</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/6ec75696-e5c1-4368-a1e6-79bb2dfa7b6d"><div class="w-full"><div class="truncate">Christian Legal Advocacy Framework</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/530b0409-4d11-471c-8249-fedc00d5ea32"><div class="w-full"><div class="truncate">Analyzing DARVO Tactics</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/1c050a53-3939-42dc-b630-9126ce05d464"><div class="w-full"><div class="truncate">Visualizing Chaotic Systems with Feigenbaum Graphs</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/06421743-853b-40e1-81ef-85f742da5c46"><div class="w-full"><div class="truncate">Hegelian Bad Faith</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/cb963dc1-461f-4347-8c25-05347bee69a6"><div class="w-full"><div class="truncate">Neural Ordinary Differential Equations</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/6f698dff-44ef-4d20-87e6-d3f8f95d7dba"><div class="w-full"><div class="truncate">Acausal Bargaining Concepts</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/9330c349-fe9a-448b-93d1-0e96af457bee"><div class="w-full"><div class="truncate">Introduction to Lambda Calculus</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/0511aaca-7e35-4b99-9727-ba35e690dc1d"><div class="w-full"><div class="truncate">Umbrella Sampling Techniques</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/c3473c1f-9ca7-47e4-9574-c4be061e5b64"><div class="w-full"><div class="truncate">Game Theory Fundamentals</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/0d5d54a6-14b4-4a56-b54c-be9455813405"><div class="w-full"><div class="truncate">Marchenko-Pastur Distribution in Random Matrix Theory</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/0e4e6a93-d45b-466e-8b1a-7e60841bb476"><div class="w-full"><div class="truncate">Assessing Intelligence Beyond IQ</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/1bb02e47-876a-4d68-9d13-e9a0c239d153"><div class="w-full"><div class="truncate">K-Metric Antidimensional Graph Theory</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/4a253f63-6dab-43e7-ab4b-aa19be46436f"><div class="w-full"><div class="truncate">Exploring Geometrodynamics</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/a632fe1f-d15e-4da4-814d-fdcec2bbd584"><div class="w-full"><div class="truncate">Quantifying Multivariate Redundancy and Synergy</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/e851db29-5cfd-4a58-9881-81180b90fdda"><div class="w-full"><div class="truncate">Fermi–Pasta–Ulam–Tsingou Problem</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">2 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/f0532063-50e5-4457-8974-6d4d2e30974a"><div class="w-full"><div class="truncate">Exploring Complex Moral Culpability</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/1b837dda-e0bf-4a05-b150-0dffab7a67ac"><div class="w-full"><div class="truncate">Degrees of Freedom and Matrix Rank Relationship</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/ca61e7b4-44cd-42af-a9ef-6a07951e753f"><div class="w-full"><div class="truncate">Kuznets Theory of Economic Development</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/313d5a59-f9d4-489a-9551-c32b78a7adaf"><div class="w-full"><div class="truncate">Formalizing Calculated Risk</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/135ac571-5f59-4985-bcc7-3a1abe7eb0f7"><div class="w-full"><div class="truncate">Flea Circus and Emergent Phenomena</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/5bc9b365-0860-4497-97af-f4450c7ada27"><div class="w-full"><div class="truncate">Synanthropic Species Adaptations</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/d01e904e-e4b3-4864-91fa-426fd1b11129"><div class="w-full"><div class="truncate">Critical Systems Thinking Methodology</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/99103af5-2117-42b8-add2-aa415c41e4d4"><div class="w-full"><div class="truncate">Hill Function Modeling</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/24d3d6a6-cc7e-47c4-89a8-81dc52e8c1c8"><div class="w-full"><div class="truncate">The Two Generals Problem</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/457683b8-4ba6-4eb1-84a0-96604d2c23d7"><div class="w-full"><div class="truncate">Phi in Integrated Information Theory</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">3 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/2244e3a5-3d75-4091-9a81-5823ae90011b"><div class="w-full"><div class="truncate">Exploring the Memory Palace Technique</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/f5278ef4-a0f7-4b09-b57a-d1033ff3a25c"><div class="w-full"><div class="truncate">Linear Regression for Binary Outcomes</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/8fb08648-b4cc-4a8e-90b4-be2bfa9d2921"><div class="w-full"><div class="truncate">Competitive Authoritarianism Explained</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/1713f33c-b073-4fa3-9381-104ec7c165b7"><div class="w-full"><div class="truncate">Quantum Hydrodynamics: Bridging Quantum Mechanics and Fluid Dynamics</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/7f086336-0adc-448a-8aad-49baa9399948"><div class="w-full"><div class="truncate">Understanding Locus of Control</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/6c777dd8-c72d-4883-9b48-697b9adf6f1e"><div class="w-full"><div class="truncate">Spliceosome Molecular Machinery</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/e3a4f018-cea4-4c8a-885c-092fefe20ec8"><div class="w-full"><div class="truncate">Granularity-Related Inconsistency of Means</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/cf3541ad-84a5-4edb-ab1b-5080edad64ef"><div class="w-full"><div class="truncate">Cretaceous Resinous Interval</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/8f40d55e-8a33-4669-848d-e36693d979f1"><div class="w-full"><div class="truncate">Rényi Entropy Concepts</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/0deacd8b-4c4e-4c3b-9f25-f132d2fc3072"><div class="w-full"><div class="truncate">Understanding Aggregation Theory</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/c070c5c2-d023-4efc-b584-890b0a667e02"><div class="w-full"><div class="truncate">William C. Wimsatt's Philosophy of Science</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">4 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/8e97c5b7-ebdd-42a1-9658-da2993fb577a"><div class="w-full"><div class="truncate">Aristarchus of Samos and the Heliocentric Model</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">5 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/b01d1f56-b956-4961-9c10-4f930adda320"><div class="w-full"><div class="truncate">Modeling as Notational Compression</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">5 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/0e9ac127-c5f2-4dea-8ac9-c0db0e61586c"><div class="w-full"><div class="truncate">Whales in Gaming and Marketing</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">5 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/34097685-a4cb-47b6-b184-9cebb982ebfa"><div class="w-full"><div class="truncate">Mechanical Advantage Through Gear Ratios</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">5 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/2fad27b3-2782-45f4-95e4-acf78aa8d661"><div class="w-full"><div class="truncate">Analyzing Engineering Failure Modes</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">5 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/85fd318f-e083-4312-b502-1753848f7b5a"><div class="w-full"><div class="truncate">Exploring Absurdist Themes in Squid Game</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">5 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/1afc567b-4bb4-432e-8111-c5186f8fe08c"><div class="w-full"><div class="truncate">Unifying Spectral Decomposition Techniques</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">6 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/7abc1704-9160-4b3a-88cf-f1287f95d894"><div class="w-full"><div class="truncate">Geopolitics of Island Nations</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">6 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/548e02c1-8c00-49d8-8cb7-e99d8f74c05c"><div class="w-full"><div class="truncate">Measures of Central Tendency</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">6 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/12715f2b-92f2-40c8-be50-bd2ce7f5296b"><div class="w-full"><div class="truncate">Inflated FreeSurfer Brain Model</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">6 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/567935de-c057-4864-95bb-aefbf2902fdd"><div class="w-full"><div class="truncate">Understanding Kalman Gain</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/de95873d-a2c7-4ce0-ac45-2f8d31d8cf2d"><div class="w-full"><div class="truncate">Dynamical Systems: Sources and Sinks</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/ee51f8b1-793d-41b5-a2e7-52ec95f4f48d"><div class="w-full"><div class="truncate">Physics as Sacred Law</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/79b3d86c-fa79-41bc-87b9-6ce279774036"><div class="w-full"><div class="truncate">Ito Process Modeling</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/757062c6-84a3-4e3a-a5ce-025caee68f62"><div class="w-full"><div class="truncate">Rich Club Network Phenomenon</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/f2daa730-72f2-4afc-ad22-fd5a93bda7ff"><div class="w-full"><div class="truncate">Tau Leaping for Stochastic Simulations</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/e0b71b76-2799-4a20-b7f3-c5f82ba51ad1"><div class="w-full"><div class="truncate">Autotelic Processes and Systems</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/84051f7e-21c3-4f0e-893d-26a017498868"><div class="w-full"><div class="truncate">Formal Modeling of Concurrent Systems</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/1310eb5f-5471-411e-85a8-249701e7b0a3"><div class="w-full"><div class="truncate">Stochastic Thermodynamics Fundamentals</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/89a0dda3-5667-4fb7-9fa7-fc583dbda261"><div class="w-full"><div class="truncate">Exploring the Cauchy-Schwarz Inequality</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/1595e60d-307e-4891-944a-738c2b794657"><div class="w-full"><div class="truncate">Critical Batch Size in Stochastic Optimization</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/2545047a-bc8c-461e-bd36-c6cb161d0273"><div class="w-full"><div class="truncate">Categorical Dimensional Analysis</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/a762a9ef-4c4c-494a-964a-a1d8ec2184ed"><div class="w-full"><div class="truncate">The Physical Church-Turing Thesis</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/4cfb88d9-f4ec-4e3f-8dad-f0b731d26452"><div class="w-full"><div class="truncate">Limits of Information in Finite Spaces</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/3cd80afe-4f42-4110-a989-afd7d7eaaf33"><div class="w-full"><div class="truncate">Designing Modern Ontologies</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/f5981a65-0e29-42ab-ad41-7909aa1e5199"><div class="w-full"><div class="truncate">Measure Theory and Radon-Nikodym Derivative</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/34b23437-5024-480e-b2d6-099a0b620355"><div class="w-full"><div class="truncate">Information Flow Across Capacity-Limited Media</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/3192c2c3-bfbf-423b-8bab-29fb984f34a0"><div class="w-full"><div class="truncate">Dimensional Analysis of Deep Learning Scaling Laws</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/d91655b3-f279-4117-9577-5640107174ac"><div class="w-full"><div class="truncate">The Energy-Information Relationship</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/b3f2ed55-b755-416d-84ec-241252872ae4"><div class="w-full"><div class="truncate">Covariance Matrix Adaptation Evolution Strategy (CMA-ES)</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/fcd0293a-6f22-427f-ab47-73387b78d53f"><div class="w-full"><div class="truncate">Analyzing Brash Communication Patterns</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/540f8300-8fd2-418f-9d35-faa4974eb239"><div class="w-full"><div class="truncate">Brash Communication Style</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/08de7241-2cd6-49a4-bc32-a58fbea5af24"><div class="w-full"><div class="truncate">Brash Communication Styles</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li><li><div class="group relative"><a class="
          border-0.5
          font-large
          flex
          cursor-pointer
          overflow-x-hidden
          text-ellipsis
          whitespace-nowrap
          rounded-xl
          bg-gradient-to-b
          py-4
          pl-5
          pr-4
          transition-all
          ease-in-out
          hover:shadow-sm
          active:scale-[0.98]
         from-bg-100 to-bg-100/30 border-border-300 hover:from-bg-000 hover:to-bg-000/80 hover:border-border-200" href="/chat/2981bcc6-6041-48de-94f8-da8457d8eeeb"><div class="w-full"><div class="truncate">Building a Rigorous AI Knowledge Base</div><div class="flex min-h-5 items-center gap-1.5"><div class="text-text-400 font-small truncate">Last message&nbsp;<span data-state="closed">7 months ago</span></div></div></div></a><button class="inline-flex
  items-center
  justify-center
  relative
  shrink-0
  can-focus
  select-none
  disabled:pointer-events-none
  disabled:opacity-50
  disabled:shadow-none
  disabled:drop-shadow-none text-text-300
          border-transparent
          transition
          font-ui
          tracking-tight
          duration-300
          ease-[cubic-bezier(0.165,0.85,0.45,1)]
          hover:bg-bg-300
          aria-pressed:bg-bg-400
          aria-checked:bg-bg-400
          aria-expanded:bg-bg-400
          hover:text-text-100
          aria-pressed:text-text-100
          aria-checked:text-text-100
          aria-expanded:text-text-100  z-2 !absolute top-2 right-2 p-2 rounded-lg transition duration-300 opacity-0 translate-x-1 ease-in-out group-hover:translate-x-0 group-hover:opacity-100 " type="button" aria-label="Delete conversation" data-state="closed"><svg xmlns="http://www.w3.org/2000/svg" width="14" height="14" fill="currentColor" viewBox="0 0 256 256"><path d="M216,48H176V40a24,24,0,0,0-24-24H104A24,24,0,0,0,80,40v8H40a8,8,0,0,0,0,16h8V208a16,16,0,0,0,16,16H192a16,16,0,0,0,16-16V64h8a8,8,0,0,0,0-16ZM96,40a8,8,0,0,1,8-8h48a8,8,0,0,1,8,8v8H96Zm96,168H64V64H192ZM112,104v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Zm48,0v64a8,8,0,0,1-16,0V104a8,8,0,0,1,16,0Z"></path></svg></button></div></li></ul>
"""

In [14]:
from bs4 import BeautifulSoup

In [15]:
soup = BeautifulSoup(project_entries.strip(), 'html.parser')

In [22]:
llm_obsidian_chats = []
anchors = soup.find_all('a')
for anchor in anchors:
    href = anchor.get('href')
    if href and href.startswith('/chat/'):
        chat_id = href.split('/')[-1]
        llm_obsidian_chats.append(chat_id)


In [23]:
len(llm_obsidian_chats)

132

In [30]:
tagger = create_default_tagger(source='claude')
tagger.add_conversation_rule('llm_obsidian_chat', lambda x: x.conversation_id in llm_obsidian_chats)
tagged_results = [tagger.tag_conversation(c) for c in convs]

In [32]:
for tr in tagged_results:
    if 'llm_obsidian_chat' in tr.annotations:
        print(f"{tr.exchange_count} - {len(tr.raw['chat_messages'])} - {tr.title}")

4 - 8 - Categorical Dimensional Analysis
1 - 2 - Stochastic Thermodynamics Fundamentals
9 - 18 - Exploring the Cauchy-Schwarz Inequality
1 - 2 - Rich Club Network Phenomenon
1 - 2 - Covariance Matrix Adaptation Evolution Strategy (CMA-ES)
4 - 8 - The Energy-Information Relationship
20 - 46 - Dimensional Analysis of Deep Learning Scaling Laws
6 - 12 - Information Flow Across Capacity-Limited Media
1 - 2 - Measure Theory and Radon-Nikodym Derivative
15 - 30 - Designing Modern Ontologies
2 - 8 - Limits of Information in Finite Spaces
2 - 4 - Critical Batch Size in Stochastic Optimization
10 - 20 - The Physical Church-Turing Thesis
2 - 4 - Formal Modeling of Concurrent Systems
2 - 4 - Autotelic Processes and Systems
2 - 4 - Tau Leaping for Stochastic Simulations
2 - 4 - Ito Process Modeling
6 - 12 - Physics as Sacred Law
8 - 16 - Dynamical Systems: Sources and Sinks
2 - 4 - Understanding Kalman Gain
2 - 4 - Inflated FreeSurfer Brain Model
1 - 2 - Building a Rigorous AI Knowledge Base
3 - 6